# Deep Continuous Bag of Words

![alt text](src/arch.jpg)

source: https://people.cs.umass.edu/~miyyer/pubs/2015_acl_dan.pdf

In [52]:
# Import necessary modules

import numpy as np
import pandas as pd
import re

from tqdm import tqdm
import random

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torch.utils.data as data_utils

In [2]:
# Create word2vec dictionary

def load_word_vectors(filename):
    word_to_index = {}
    word_vectors  = []
    
    with open(filename, encoding="utf8") as fp:
        for line in tqdm(fp.readlines(), leave = False):
            line = line.split(" ")
            
            word = line[0]
            word_to_index[word] = len(word_to_index)
            
            vec = np.array([float(x) for x in line[1:]])
            word_vectors.append(vec)
            
    return word_to_index, word_vectors

word_to_index, word_vectors = load_word_vectors("model/glove.6B.100d.txt")

In [3]:
# Import csv file of amazon reviews

df = pd.read_csv("data/amazon_reviews_small.csv", names = ['sentiment', 'title', 'content'])
df.head()

,sentiment,title,content
0,2,Right on the money,We are using the this book to get 100+ certifi...
1,2,Serves its Purpose!,Couldn't go without it. My 3 1/2 year still we...
2,2,Trailer Park Bwoys!!!,we get to see it on paramount in ol' LND UK an...
3,1,buyer beware,There are companies selling Bosch knock-offs o...
4,2,Great for those cold winters,If you are looking to keep your water liquifie...


In [4]:
def get_vectors(phrase):
    phrase = phrase.split(' ')
    vector = np.zeros((1,100))
    word_count = 0
    for word in phrase:
        try:
            vector += np.asarray(word_vectors[word_to_index[word]])
            word_count += 1
        except:
            pass
    
    if word_count > 0:
        ans = vector/word_count
    else:
        ans = np.zeros((1,100))
    
    return ans
    
def clean_phrase(phrase):
    phrase = phrase.lower()
    phrase = phrase.replace("'", "")
    phrase = phrase.replace(".", "")
    phrase = re.sub(r'([^\s\w]|_)+', '', phrase)
    return phrase

def generate_phrases(dataset):
    phrases = dataset[['title', 'content']].dropna().values.flatten().tolist()
    return phrases
    
def generate_tfidf(phrases):
    tfidf = TfidfVectorizer()
    tfidf.fit(phrases)
    return tfidf

def phrase_cbow(a, b):
    aa = a.reshape(1,-1)
    ba = b.reshape(1,-1)
    
    cos_lib = cosine_similarity(aa, ba)

    return cos_lib

In [5]:
df['sentiment']     = [(a-1) for a in df.sentiment.values]
df['title_clean']   = [clean_phrase(str(a)) for a in df.title]
df['content_clean'] = [clean_phrase(str(a)) for a in df.content]

In [6]:
tfidf = generate_tfidf(generate_phrases(df))

In [7]:
print("Generating clean title")
df['title_rep']     = [tfidf.transform([a]) for a in df.title_clean]
print("Generating clean content")
df['content_rep']   = [tfidf.transform([a]) for a in df.content_clean]

Generating clean title
Generating clean content


In [8]:
print("Generating clean title vector")
df['title_vec']     = [get_vectors(a) for a in df.title_clean]
print("Generating clean content vector")
df['content_vec']   = [get_vectors(a) for a in df.content_clean]

Generating clean title vector
Generating clean content vector


In [9]:
df['cosine']        = [phrase_cbow(get_vectors(a), get_vectors(b)) for a,b in zip(df.title_clean, df.content_clean)]

In [10]:
df['feature'] = [np.hstack((a,b,c)).reshape(1,-1) for a,b,c in df[['title_vec', 'content_vec', 'cosine']].values]

In [11]:
df_set = df.drop(columns = ['title_clean', 'content_clean', 'title_rep', 'content_rep', 'title_vec', 'content_vec', 'cosine'])
df_set.head()

,sentiment,title,content,feature
0,1,Right on the money,We are using the this book to get 100+ certifi...,"[[0.11201899999999998, 0.03991249999999999, 0...."
1,1,Serves its Purpose!,Couldn't go without it. My 3 1/2 year still we...,"[[-0.08785333333333334, -0.022782999999999998,..."
2,1,Trailer Park Bwoys!!!,we get to see it on paramount in ol' LND UK an...,"[[0.23044, -0.04867500000000001, 0.431965, -0...."
3,0,buyer beware,There are companies selling Bosch knock-offs o...,"[[0.10213000000000001, 0.18035, 0.790140000000..."
4,1,Great for those cold winters,If you are looking to keep your water liquifie...,"[[-0.3699188, 0.3999688, 0.37708379999999997, ..."


In [132]:
pos_sent = df.loc[df['sentiment'] == 1]
neg_sent = df.loc[df['sentiment'] != 1]

pos_sent['sentiment'] = [[0, 1] for a in pos_sent['sentiment'].values]
neg_sent['sentiment'] = [[1, 0] for a in neg_sent['sentiment'].values]

pos_temp = pos_sent.sample(frac = 1)
neg_temp = neg_sent.sample(frac = 1)

pos_points = len(pos_temp)
neg_points = len(neg_temp)

train_pos = pos_temp.iloc[:int(pos_points*.7)]
train_neg = neg_temp.iloc[:int(neg_points*.7)]
train     = pd.concat([train_pos, train_neg])

valid_pos = pos_temp.iloc[int(pos_points*.7):int(pos_points*.85)]
valid_neg = neg_temp.iloc[int(neg_points*.7):int(neg_points*.85)]
valid     = pd.concat([valid_pos, valid_neg])

test_pos  = pos_temp.iloc[int(pos_points*.85):]
test_neg  = neg_temp.iloc[int(neg_points*.85):]
test      = pd.concat([test_pos, test_neg])

#pd.concat([a, b])

D:\Users\Prince\Anaconda3\envs\pytorch\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
D:\Users\Prince\Anaconda3\envs\pytorch\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """


In [13]:
np.concatenate(np.asarray(train.feature.values))

array([[-2.72975000e-01,  2.08066500e-01,  2.04910000e-01, ...,
         4.70023125e-01,  3.13645115e-01,  8.06552182e-01],
       [ 7.60000000e-04,  7.40733333e-02,  5.50811167e-01, ...,
         3.74438457e-01,  2.44254436e-01,  8.97158113e-01],
       [ 2.03083333e-02,  3.86762000e-01,  3.46700000e-01, ...,
         6.08676554e-01,  2.32992838e-01,  8.81940123e-01],
       ...,
       [-1.00409692e-01,  1.78799077e-01,  4.53550385e-01, ...,
         2.67125359e-01,  3.42334231e-01,  9.69899721e-01],
       [-3.09985000e-01, -8.03162500e-02,  5.36435000e-04, ...,
         5.25560185e-01,  7.80769778e-02,  8.80850007e-01],
       [-1.07940000e-02, -2.61457500e-02,  4.28662500e-01, ...,
         5.14350984e-01,  9.80330588e-02,  7.30414923e-01]])

In [14]:
train_tensor = data_utils.TensorDataset(torch.tensor(np.concatenate(np.asarray(train.feature.values), axis = 0)).float(), 
                                        torch.tensor(np.vstack(train.sentiment.values)))
valid_tensor = data_utils.TensorDataset(torch.tensor(np.concatenate(np.asarray(valid.feature.values), axis = 0)).float(), 
                                        torch.tensor(np.vstack(valid.sentiment.values)))
test_tensor  = data_utils.TensorDataset(torch.tensor(np.concatenate(np.asarray(test.feature.values), axis = 0)).float(), 
                                        torch.tensor(np.vstack(test.sentiment.values)))

In [15]:
batch_size = 32

train_loader = data_utils.DataLoader(train_tensor, batch_size = batch_size, shuffle = True)
valid_loader = data_utils.DataLoader(valid_tensor, batch_size = batch_size, shuffle = True)
test_loader = data_utils.DataLoader(test_tensor, batch_size = batch_size, shuffle = True)

In [16]:
train_on_gpu = torch.cuda.is_available()

model = nn.Sequential(nn.Linear(201, 100),
                      nn.ReLU(),
                      nn.Linear(100, 50),
                      nn.ReLU(),
                      nn.Linear(50, 2),
                      nn.Softmax(dim=1))

if train_on_gpu:
    model.cuda()

criterion = nn.BCELoss()
optimizer = optim.SGD(model.parameters(), lr=0.03)

In [17]:
epochs = 10
valid_min = np.Inf
for e in range(epochs):
    running_loss_train = 0
    model.train
    for features, labels in train_loader:
        if train_on_gpu:
            features, labels = features.cuda(), labels.cuda()
        optimizer.zero_grad()
        output = model(features)
        loss = criterion(output, labels.float())
        loss.backward()
        optimizer.step()
        
        running_loss_train += loss.item()

        
    running_loss_valid = 0
    model.eval
    for features, labels in train_loader:
        if train_on_gpu:
            features, labels = features.cuda(), labels.cuda()
        output = model(features)
        loss = criterion(output, labels.float())
        
        running_loss_valid += loss.item()

        
    print(f"Epoch {e}    Training loss: {running_loss_train/len(train_loader):.6f}\tValidation loss: {running_loss_valid/len(train_loader):.6f}")
    
    if running_loss_valid < valid_min:
        print("Validation loss decreased. Saving model...")
        torch.save(model.state_dict(), 'model.pt')
        valid_min = running_loss_valid

Epoch 0    Training loss: 0.504370	Validation loss: 0.490477
Validation loss decreased. Saving model...
Epoch 1    Training loss: 0.421807	Validation loss: 0.393432
Validation loss decreased. Saving model...
Epoch 2    Training loss: 0.403042	Validation loss: 0.381404
Validation loss decreased. Saving model...
Epoch 3    Training loss: 0.392264	Validation loss: 0.372416
Validation loss decreased. Saving model...
Epoch 4    Training loss: 0.380691	Validation loss: 0.359096
Validation loss decreased. Saving model...
Epoch 5    Training loss: 0.371665	Validation loss: 0.412038
Epoch 6    Training loss: 0.365496	Validation loss: 0.347995
Validation loss decreased. Saving model...
Epoch 7    Training loss: 0.359001	Validation loss: 0.355871
Epoch 8    Training loss: 0.352776	Validation loss: 0.347332
Validation loss decreased. Saving model...
Epoch 9    Training loss: 0.348699	Validation loss: 0.375379


In [28]:
test_loss = 0.0
model.eval

model.load_state_dict(torch.load('model.pt'))

correct = 0
total = 0

model.cuda()

for features, labels in test_loader:
    if train_on_gpu:
        features, labels = features.cuda(), labels.cuda()
    output = model(features)
    print(output, labels)
    _, pred = torch.max(output, 1)
    _, true = torch.max(labels, 1)
    loss = criterion(output, labels.float())
    test_loss += loss.item()
    for a, b in zip(pred, true):
        if a.item() == b.item():
            correct += 1
        total += 1

print(f"Model test loss is {test_loss/len(test_loader):.6f}.")
print(f"Model got {correct} out of {total} correct.")
print(f"Model test accuracy: {correct*100/total:.2f}%")

tensor([[0.0281, 0.9719],
        [0.9143, 0.0857],
        [0.9549, 0.0451],
        [0.0204, 0.9796],
        [0.9851, 0.0149],
        [0.9971, 0.0029],
        [0.9959, 0.0041],
        [0.9190, 0.0810],
        [0.9348, 0.0652],
        [0.0230, 0.9770],
        [0.4016, 0.5984],
        [0.9138, 0.0862],
        [0.0228, 0.9772],
        [0.1601, 0.8399],
        [0.9214, 0.0786],
        [0.9339, 0.0661],
        [0.9578, 0.0422],
        [0.9623, 0.0377],
        [0.0025, 0.9975],
        [0.9506, 0.0494],
        [0.0431, 0.9569],
        [0.0972, 0.9028],
        [0.0167, 0.9833],
        [0.6669, 0.3331],
        [0.2463, 0.7537],
        [0.9109, 0.0891],
        [0.8822, 0.1178],
        [0.8692, 0.1308],
        [0.0733, 0.9267],
        [0.9682, 0.0318],
        [0.3944, 0.6056],
        [0.9961, 0.0039]], device='cuda:0', grad_fn=<SoftmaxBackward>) tensor([[0, 1],
        [1, 0],
        [1, 0],
        [0, 1],
        [0, 1],
        [1, 0],
        [1, 0],
        [1,

        [8.2839e-01, 1.7161e-01]], device='cuda:0', grad_fn=<SoftmaxBackward>) tensor([[0, 1],
        [1, 0],
        [1, 0],
        [1, 0],
        [1, 0],
        [0, 1],
        [1, 0],
        [1, 0],
        [1, 0],
        [1, 0],
        [0, 1],
        [0, 1],
        [1, 0],
        [1, 0],
        [0, 1],
        [1, 0],
        [0, 1],
        [1, 0],
        [0, 1],
        [0, 1],
        [0, 1],
        [1, 0],
        [0, 1],
        [1, 0],
        [0, 1],
        [0, 1],
        [1, 0],
        [0, 1],
        [1, 0],
        [1, 0],
        [0, 1],
        [1, 0]], device='cuda:0', dtype=torch.int32)
tensor([[8.5264e-01, 1.4736e-01],
        [1.1572e-01, 8.8428e-01],
        [1.5073e-01, 8.4927e-01],
        [9.8315e-02, 9.0168e-01],
        [9.5574e-01, 4.4256e-02],
        [3.6938e-01, 6.3062e-01],
        [1.1151e-01, 8.8849e-01],
        [4.3913e-01, 5.6087e-01],
        [7.5969e-01, 2.4031e-01],
        [9.8908e-01, 1.0923e-02],
        [9.1698e-01, 8.3020e-02]

        [1, 0]], device='cuda:0', dtype=torch.int32)
tensor([[1.6303e-01, 8.3697e-01],
        [6.5958e-01, 3.4042e-01],
        [4.6025e-02, 9.5398e-01],
        [7.5804e-01, 2.4196e-01],
        [5.6178e-01, 4.3822e-01],
        [9.1329e-01, 8.6714e-02],
        [7.0027e-01, 2.9973e-01],
        [2.9164e-01, 7.0836e-01],
        [9.1180e-01, 8.8199e-02],
        [1.4081e-01, 8.5919e-01],
        [2.8728e-01, 7.1272e-01],
        [9.3336e-01, 6.6642e-02],
        [4.6703e-03, 9.9533e-01],
        [9.4286e-01, 5.7144e-02],
        [5.2149e-02, 9.4785e-01],
        [9.6509e-01, 3.4905e-02],
        [2.6275e-02, 9.7373e-01],
        [2.3016e-02, 9.7698e-01],
        [2.7744e-02, 9.7226e-01],
        [8.5008e-01, 1.4992e-01],
        [6.5562e-01, 3.4438e-01],
        [7.4129e-03, 9.9259e-01],
        [6.4790e-01, 3.5210e-01],
        [2.0231e-01, 7.9769e-01],
        [6.1093e-02, 9.3891e-01],
        [9.6836e-01, 3.1639e-02],
        [9.9940e-01, 6.0159e-04],
        [4.4358e-01, 5.5642e-

        [9.9659e-01, 3.4070e-03]], device='cuda:0', grad_fn=<SoftmaxBackward>) tensor([[0, 1],
        [0, 1],
        [0, 1],
        [0, 1],
        [0, 1],
        [1, 0],
        [0, 1],
        [0, 1],
        [1, 0],
        [1, 0],
        [0, 1],
        [0, 1],
        [1, 0],
        [1, 0],
        [0, 1],
        [0, 1],
        [0, 1],
        [0, 1],
        [1, 0],
        [1, 0],
        [0, 1],
        [0, 1],
        [1, 0],
        [0, 1],
        [0, 1],
        [0, 1],
        [0, 1],
        [0, 1],
        [0, 1],
        [1, 0],
        [1, 0],
        [1, 0]], device='cuda:0', dtype=torch.int32)
tensor([[0.9948, 0.0052],
        [0.7166, 0.2834],
        [0.9453, 0.0547],
        [0.3892, 0.6108],
        [0.5778, 0.4222],
        [0.9268, 0.0732],
        [0.0480, 0.9520],
        [0.4906, 0.5094],
        [0.3122, 0.6878],
        [0.4781, 0.5219],
        [0.7905, 0.2095],
        [0.9531, 0.0469],
        [0.8785, 0.1215],
        [0.3178, 0.6822],
        

        [1.2369e-02, 9.8763e-01]], device='cuda:0', grad_fn=<SoftmaxBackward>) tensor([[1, 0],
        [0, 1],
        [1, 0],
        [0, 1],
        [0, 1],
        [0, 1],
        [0, 1],
        [1, 0],
        [0, 1],
        [0, 1],
        [1, 0],
        [1, 0],
        [0, 1],
        [1, 0],
        [0, 1],
        [1, 0],
        [0, 1],
        [1, 0],
        [1, 0],
        [1, 0],
        [0, 1],
        [0, 1],
        [1, 0],
        [1, 0],
        [1, 0],
        [1, 0],
        [0, 1],
        [1, 0],
        [1, 0],
        [1, 0],
        [0, 1],
        [0, 1]], device='cuda:0', dtype=torch.int32)
tensor([[0.3658, 0.6342],
        [0.7853, 0.2147],
        [0.9391, 0.0609],
        [0.9328, 0.0672],
        [0.8095, 0.1905],
        [0.7089, 0.2911],
        [0.0136, 0.9864],
        [0.9942, 0.0058],
        [0.7054, 0.2946],
        [0.5434, 0.4566],
        [0.5961, 0.4039],
        [0.8571, 0.1429],
        [0.0481, 0.9519],
        [0.9892, 0.0108],
        

...

        [1, 0]], device='cuda:0', dtype=torch.int32)
tensor([[0.0288, 0.9712],
        [0.8766, 0.1234],
        [0.8312, 0.1688],
        [0.9343, 0.0657],
        [0.5921, 0.4079],
        [0.6937, 0.3063],
        [0.2143, 0.7857],
        [0.7627, 0.2373],
        [0.3518, 0.6482],
        [0.9779, 0.0221],
        [0.2912, 0.7088],
        [0.2344, 0.7656],
        [0.0823, 0.9177],
        [0.8428, 0.1572],
        [0.8809, 0.1191],
        [0.9753, 0.0247],
        [0.8353, 0.1647],
        [0.9291, 0.0709],
        [0.1047, 0.8953],
        [0.2656, 0.7344],
        [0.1435, 0.8565],
        [0.2604, 0.7396],
        [0.9670, 0.0330],
        [0.9009, 0.0991],
        [0.7863, 0.2137],
        [0.8695, 0.1305],
        [0.9585, 0.0415],
        [0.8916, 0.1084],
        [0.0594, 0.9406],
        [0.0575, 0.9425],
        [0.8149, 0.1851],
        [0.9601, 0.0399]], device='cuda:0', grad_fn=<SoftmaxBackward>) tensor([[0, 1],
        [1, 0],
        [1, 0],
        [1, 0],
      

        [0.5775, 0.4225]], device='cuda:0', grad_fn=<SoftmaxBackward>) tensor([[1, 0],
        [1, 0],
        [1, 0],
        [0, 1],
        [0, 1],
        [1, 0],
        [1, 0],
        [0, 1],
        [1, 0],
        [0, 1],
        [0, 1],
        [0, 1],
        [1, 0],
        [0, 1],
        [0, 1],
        [1, 0],
        [1, 0],
        [1, 0],
        [1, 0],
        [0, 1],
        [0, 1],
        [0, 1],
        [0, 1],
        [0, 1],
        [0, 1],
        [0, 1],
        [1, 0],
        [0, 1],
        [1, 0],
        [1, 0],
        [0, 1],
        [0, 1]], device='cuda:0', dtype=torch.int32)
tensor([[0.0882, 0.9118],
        [0.0060, 0.9940],
        [0.1794, 0.8206],
        [0.1750, 0.8250],
        [0.4268, 0.5732],
        [0.1657, 0.8343],
        [0.1866, 0.8134],
        [0.5366, 0.4634],
        [0.2827, 0.7173],
        [0.9159, 0.0841],
        [0.8568, 0.1432],
        [0.0315, 0.9685],
        [0.9692, 0.0308],
        [0.9861, 0.0139],
        [0.5467,

        [4.3082e-02, 9.5692e-01]], device='cuda:0', grad_fn=<SoftmaxBackward>) tensor([[1, 0],
        [0, 1],
        [0, 1],
        [0, 1],
        [0, 1],
        [1, 0],
        [0, 1],
        [0, 1],
        [1, 0],
        [0, 1],
        [0, 1],
        [0, 1],
        [1, 0],
        [0, 1],
        [1, 0],
        [1, 0],
        [1, 0],
        [1, 0],
        [0, 1],
        [1, 0],
        [0, 1],
        [1, 0],
        [0, 1],
        [1, 0],
        [1, 0],
        [1, 0],
        [1, 0],
        [0, 1],
        [1, 0],
        [1, 0],
        [1, 0],
        [0, 1]], device='cuda:0', dtype=torch.int32)
tensor([[9.4016e-01, 5.9836e-02],
        [9.8523e-01, 1.4766e-02],
        [9.4928e-01, 5.0718e-02],
        [9.4018e-01, 5.9817e-02],
        [2.3710e-01, 7.6290e-01],
        [8.2850e-01, 1.7150e-01],
        [3.7452e-01, 6.2548e-01],
        [9.6202e-01, 3.7977e-02],
        [9.1142e-01, 8.8578e-02],
        [7.3383e-02, 9.2662e-01],
        [2.8281e-01, 7.1719e-01]

        [0, 1]], device='cuda:0', dtype=torch.int32)
tensor([[0.6861, 0.3139],
        [0.1792, 0.8208],
        [0.1172, 0.8828],
        [0.0580, 0.9420],
        [0.0510, 0.9490],
        [0.8729, 0.1271],
        [0.0994, 0.9006],
        [0.5159, 0.4841],
        [0.0511, 0.9489],
        [0.9872, 0.0128],
        [0.4601, 0.5399],
        [0.7327, 0.2673],
        [0.7306, 0.2694],
        [0.8521, 0.1479],
        [0.9287, 0.0713],
        [0.8679, 0.1321],
        [0.0602, 0.9398],
        [0.6661, 0.3339],
        [0.0278, 0.9722],
        [0.1629, 0.8371],
        [0.9226, 0.0774],
        [0.0395, 0.9605],
        [0.9415, 0.0585],
        [0.9331, 0.0669],
        [0.0090, 0.9910],
        [0.9309, 0.0691],
        [0.0621, 0.9379],
        [0.9010, 0.0990],
        [0.1293, 0.8707],
        [0.8423, 0.1577],
        [0.9768, 0.0232],
        [0.0084, 0.9916]], device='cuda:0', grad_fn=<SoftmaxBackward>) tensor([[0, 1],
        [0, 1],
        [0, 1],
        [1, 0],
      

        [7.2944e-01, 2.7056e-01]], device='cuda:0', grad_fn=<SoftmaxBackward>) tensor([[0, 1],
        [1, 0],
        [1, 0],
        [0, 1],
        [1, 0],
        [1, 0],
        [1, 0],
        [1, 0],
        [0, 1],
        [0, 1],
        [0, 1],
        [0, 1],
        [0, 1],
        [1, 0],
        [1, 0],
        [1, 0],
        [1, 0],
        [1, 0],
        [0, 1],
        [1, 0],
        [1, 0],
        [0, 1],
        [1, 0],
        [0, 1],
        [0, 1],
        [0, 1],
        [1, 0],
        [0, 1],
        [1, 0],
        [0, 1],
        [1, 0],
        [0, 1]], device='cuda:0', dtype=torch.int32)
tensor([[0.2387, 0.7613],
        [0.7423, 0.2577],
        [0.0816, 0.9184],
        [0.7352, 0.2648],
        [0.9772, 0.0228],
        [0.9065, 0.0935],
        [0.6004, 0.3996],
        [0.0162, 0.9838],
        [0.6145, 0.3855],
        [0.0486, 0.9514],
        [0.1649, 0.8351],
        [0.9309, 0.0691],
        [0.9988, 0.0012],
        [0.4380, 0.5620],
        

In [133]:
model.cpu()
def predict(index, model, tfidf):
    title = pos_sent['title'].iloc[index]
    content = pos_sent['content'].iloc[index]
    
    a = tfidf.transform([title])
    b = tfidf.transform([content])
    sim = phrase_cbow(a,b)
    a_vec = get_vectors(title)
    b_vec = get_vectors(content)
    
    feature = np.hstack((np.asarray(a_vec), sim, np.asarray(b_vec)))
    output = model(torch.tensor(feature).float())
    _, pred = torch.max(output, 1)
    
    print(f"Title: {title}")
    print(f"Content: \n{content}\n")
    
    if pred == 1:
        prediction = "Positive review"
    else:
        prediction = "Negative review"
        
    if df['sentiment'].iloc[index] == 1:
        truth = "Positive review"
    else:
        truth = "Negative review"
        
    print(f"Prediciton:\t{prediction}\nGround Truth:\t{truth}")
    print(f"Model is {'correct' if prediction == truth else 'incorrect'}.")

In [134]:
predict(a, model, tfidf)

Title: Great for Family
Content: 
This game is great for introducing kids to more interesting board games after they've already learned how to take turns. I can play with my 4 and 6 year old and not get bored, though towards the end of the game sometimes they will. Reducing the number of pawns and letting one start helps shorten the game. The pieces, board, and box are well made. It is good to have the kids understand counting, the strategy of which pawns to move, and also that they shouldn't get upset when they are bumped back (because they'll be able to do it to someone else too).

Prediciton:	Negative review
Ground Truth:	Positive review
Model is incorrect.
